# AI21 Agent with LangGraph and Tavily

This notebook demonstrates how to create an AI21 agent that uses LangGraph to execute Tavily API to search for information in the internet. 

To get Tavili API key, you need to sign up at [Tavily](https://tavily.com/).

In [ ]:
import os

from langchain_ai21 import ChatAI21
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, StateGraph, START
import operator
from typing import Annotated, TypedDict, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]


os.environ["TAVILY_API_KEY"] = "tvly-api-key"
os.environ["AI21_API_KEY"] = "ai21-api-key"

tools = [TavilySearchResults(max_results=5)]
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatAI21(model="jamba-1.5-large", temperature=0)
agent_runnable = create_tool_calling_agent(llm, tools, prompt=prompt)

tool_executor = ToolNode(tools)

def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

def execute_tools(data):
    outcomes = []
    for agent_action in data["agent_outcome"]:
        output = tool_executor.invoke(agent_action.message_log)
        outcomes.append((agent_action, str(output)))
    return {"intermediate_steps": outcomes}

def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

workflow.add_edge("action", "agent")

app = workflow.compile()

inputs = {
    "input": "Research and compile a comprehensive report on the current state of artificial intelligence "
             "in healthcare, highlighting the top companies and their innovative solutions.",
    "chat_history": []
}

for s in app.stream(inputs):
    print(list(s.values())[0])
    print("-----------------")